<a href="https://colab.research.google.com/github/ZachPetroff/Regression/blob/main/regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import zipfile

In [ ]:
train_df = pd.read_csv("../input/nyc-taxi-trip-duration/train.zip", compression="zip")
test_df = pd.read_csv("../input/nyc-taxi-trip-duration/test.zip", compression="zip")
train_df.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435


In [ ]:
train_df.describe()

,vendor_id,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration
count,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06
mean,1.534950e+00,1.664530e+00,-7.397349e+01,4.075092e+01,-7.397342e+01,4.075180e+01,9.594923e+02
std,4.987772e-01,1.314242e+00,7.090186e-02,3.288119e-02,7.064327e-02,3.589056e-02,5.237432e+03
min,1.000000e+00,0.000000e+00,-1.219333e+02,3.435970e+01,-1.219333e+02,3.218114e+01,1.000000e+00
25%,1.000000e+00,1.000000e+00,-7.399187e+01,4.073735e+01,-7.399133e+01,4.073588e+01,3.970000e+02
50%,2.000000e+00,1.000000e+00,-7.398174e+01,4.075410e+01,-7.397975e+01,4.075452e+01,6.620000e+02
75%,2.000000e+00,2.000000e+00,-7.396733e+01,4.076836e+01,-7.396301e+01,4.076981e+01,1.075000e+03
max,2.000000e+00,9.000000e+00,-6.133553e+01,5.188108e+01,-6.133553e+01,4.392103e+01,3.526282e+06


In [ ]:
test_df = test_df.drop(["id"], axis=1)
train_df = train_df.drop(["id", "dropoff_datetime"], axis=1) # dropping drop off time, because having the pickup and dropoff times would be cheating

In [ ]:
print(train_df.isna().sum())

vendor_id             0
pickup_datetime       0
passenger_count       0
pickup_longitude      0
pickup_latitude       0
dropoff_longitude     0
dropoff_latitude      0
store_and_fwd_flag    0
trip_duration         0
dtype: int64


In [ ]:
# get dummy variables for categorical data, vendor id and store and fwd flag

flag_dummies = pd.get_dummies(train_df["store_and_fwd_flag"])
vendor_dummies = pd.get_dummies(train_df["vendor_id"])

train_df = train_df.drop(["vendor_id", "store_and_fwd_flag"], axis=1)
train_df = pd.concat([train_df, flag_dummies["N"]], axis=1)
train_df = pd.concat([train_df, vendor_dummies[2]], axis=1)

test_flag_dummies = pd.get_dummies(test_df["store_and_fwd_flag"])
test_vendor_dummies = pd.get_dummies(test_df["vendor_id"])

test_df = test_df.drop(["vendor_id", "store_and_fwd_flag"], axis=1)
test_df = pd.concat([test_df, test_flag_dummies["N"]], axis=1)
test_df = pd.concat([test_df, test_vendor_dummies[2]], axis=1)

In [ ]:
morning = []
midday = []
night = []

summer = []
winter = []
spring = [] 
fall = []

morning_test = []
midday_test = []
night_test = []

summer_test = []
winter_test = []
spring_test = []
fall_test = []

dates = train_df["pickup_datetime"]
test_dates = test_df["pickup_datetime"]

for i in range(len(dates)):
    date = dates.iloc[i]
    if i < len(test_dates):
        test_date = test_dates.iloc[i]
    
    month = date[5:7]
    hour = date[11:13]
    
    if i < len(test_dates):
        test_month = test_date[5:7]
        test_hour = test_date[11:13]
    
    month = int(month)
    hour = int(hour)
    
    if i < len(test_dates):
        test_month = int(test_month)
        test_hour = int(test_hour)

    if month == 12 or month <= 2:
        winter.append(1)
        summer.append(0)
        spring.append(0)
        fall.append(0)
        if i < len(test_dates):
            winter_test.append(1)
            summer_test.append(0)
            spring_test.append(0)
            fall_test.append(0)
    if 2 < month and month <= 5:
        spring.append(1)
        summer.append(0)
        fall.append(0)
        winter.append(0)
        if i < len(test_dates):
            spring_test.append(1)
            summer_test.append(0)
            fall_test.append(0)
            winter_test.append(0)
    if 5 < month and month <= 8:
        summer.append(1)
        spring.append(0)
        fall.append(0)
        winter.append(0)
        if i < len(test_dates):
            summer_test.append(1)
            spring_test.append(0)
            fall_test.append(0)
            winter_test.append(0)
    if 8 < month and month <= 11:
        fall.append(1)
        summer.append(0)
        spring.append(0)
        winter.append(0)
        if i < len(test_dates):
            fall_test.append(1)
            summer_test.append(0)
            spring_test.append(0)
            winter_test.append(0)
    if 4 <= hour and hour < 12:
        morning.append(1)
        midday.append(0)
        night.append(0)
        if i < len(test_dates):
            morning_test.append(1)
            midday_test.append(0)
            night_test.append(0)
    if 12 <= hour and hour < 20:
        midday.append(1)
        morning.append(0)
        night.append(0)
        if i < len(test_dates):
            midday_test.append(1)
            morning_test.append(0)
            night_test.append(0)
    if hour < 4 or hour >= 20:
        night.append(1)
        morning.append(0)
        midday.append(0)
        if i < len(test_dates):
            night_test.append(1)
            morning_test.append(0)
            midday_test.append(0)

In [ ]:
train_df.insert(1, "morning", morning)
train_df.insert(1, "midday", midday)
train_df.insert(1, "night", night)
train_df.insert(1, "summer", summer)
train_df.insert(1, "winter", winter)
train_df.insert(1, "spring", spring)
train_df.insert(1, "fall", fall)

test_df.insert(1, "morning", morning_test)
test_df.insert(1, "midday", midday_test)
test_df.insert(1, "night", night_test)
test_df.insert(1, "summer", summer_test)
test_df.insert(1, "winter", winter_test)
test_df.insert(1, "spring", spring_test)
test_df.insert(1, "fall", fall_test)

In [ ]:
train_df = train_df.drop(["pickup_datetime"], axis=1)
test_df = test_df.drop(["pickup_datetime"], axis=1)

In [ ]:
X = train_df.drop(["trip_duration"], axis=1)
y = train_df["trip_duration"]

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

regr = RandomForestRegressor(max_depth=4, random_state=0)
pipe = Pipeline([('scaler', StandardScaler()), ('random forest', regr)])

pipe.fit(X, y)

Pipeline(steps=[('scaler', StandardScaler()),
                ('random forest',
                 RandomForestRegressor(max_depth=4, random_state=0))])

In [ ]:
importances = regr.feature_importances_

In [ ]:
feature_names = X.columns
imp_dict = {}
for i in range(len(feature_names)):
    imp_dict[feature_names[i]] = importances[i]

In [ ]:
dict(sorted(imp_dict.items(), key=lambda item: item[1]))

{'fall': 0.0,
 'morning': 0.0,
 'N': 0.0,
 'winter': 3.7060944620407174e-11,
 'spring': 0.00026031051370101416,
 'summer': 0.00037131485711386586,
 'midday': 0.001812225064189105,
 'passenger_count': 0.0075470198169940725,
 'night': 0.0121195757628792,
 2: 0.030108462944862066,
 'dropoff_latitude': 0.0760748463493562,
 'pickup_latitude': 0.1764723024012573,
 'pickup_longitude': 0.31241017416846645,
 'dropoff_longitude': 0.3828237680841197}

In [ ]:
# dropping features of little or no importance
X = X.drop(['fall', 'morning', 'N', 'winter'], axis=1)
test_df = test_df.drop(['fall', 'morning', 'N', 'winter'], axis=1)

In [ ]:
X.head()

,spring,summer,night,midday,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,2
0,1,0,0,1,1,-73.982155,40.767937,-73.964630,40.765602,1
1,0,1,1,0,1,-73.980415,40.738564,-73.999481,40.731152,0
2,0,0,0,0,1,-73.979027,40.763939,-74.005333,40.710087,1
3,1,0,0,1,1,-74.010040,40.719971,-74.012268,40.706718,1
4,1,0,0,1,1,-73.973053,40.793209,-73.972923,40.782520,1


In [ ]:
''' I thought about the how the data might relate to the target, particularly linear relationships,
    and I realized that the pickup and drop off locations themselves probably do not have a good linear
    relationship with the trip duration, but we can use these values to find the distance of the trip, 
    which likely has a really strong linear relationship with the duration of the trip. 
    I test this theory below.
'''

'''
def euclidean_dist(x1, x2, y1, y2):
    return ((x2 - x1) ** 2) + ((y2 - y1) ** 2)

dists = []
for i in range(len(X)):
    dist = euclidean_dist(X["pickup_longitude"][i], X["dropoff_longitude"][i], X["pickup_latitude"][i], X["dropoff_latitude"][i])
    dists.append(dist)
    
dists = np.array(dists)
durations = y.to_numpy()

np.corrcoef(dists, durations)[0][1]
'''
# There is very little correlation, my theory was incorrect 

'\ndef euclidean_dist(x1, x2, y1, y2):\n    return ((x2 - x1) ** 2) + ((y2 - y1) ** 2)\n\ndists = []\nfor i in range(len(X)):\n    dist = euclidean_dist(X["pickup_longitude"][i], X["dropoff_longitude"][i], X["pickup_latitude"][i], X["dropoff_latitude"][i])\n    dists.append(dist)\n    \ndists = np.array(dists)\ndurations = y.to_numpy()\n\nnp.corrcoef(dists, durations)[0][1]\n'

In [ ]:
from sklearn.svm import LinearSVR
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.kernel_approximation import Nystroem

'''
gammas = [.01, .1, 1, 10]
comps = [100, 200, 300]

for i in range(len(gammas)):
    for j in range(len(comps)):
        feature_map_nystroem = Nystroem(gamma=gammas[i],
                                random_state=1,
                                n_components=comps[j])
        data_transformed = feature_map_nystroem.fit_transform(X)
        regr = make_pipeline(StandardScaler(), LinearSVR(random_state=0, tol=1e-5))
        score = cross_val_score(regr, data_transformed, y, cv=3, scoring = 'r2').mean()
        
        print("Gamma = ", gammas[i], " |N Components = ", comps[j], " |Score = ", score)
'''

'\ngammas = [.01, .1, 1, 10]\ncomps = [100, 200, 300]\n\nfor i in range(len(gammas)):\n    for j in range(len(comps)):\n        feature_map_nystroem = Nystroem(gamma=gammas[i],\n                                random_state=1,\n                                n_components=comps[j])\n        data_transformed = feature_map_nystroem.fit_transform(X)\n        regr = make_pipeline(StandardScaler(), LinearSVR(random_state=0, tol=1e-5))\n        score = cross_val_score(regr, data_transformed, y, cv=3, scoring = \'r2\').mean()\n        \n        print("Gamma = ", gammas[i], " |N Components = ", comps[j], " |Score = ", score)\n'

In [ ]:
# Use random forest regressor, ridge regression, lasso regression, support vector regression, XGBoost Regression, and Elastic net
# Find the best performing model, then fine tune
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
from sklearn.kernel_ridge import KernelRidge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_val_score

'''
forest = RandomForestRegressor(max_depth=3, random_state=0)
kernel = KernelRidge()
elastic = ElasticNet(random_state=0)
ridge = Ridge(random_state=0)
lasso = Lasso(random_state=0)
neigh = KNeighborsRegressor(n_neighbors=2)

models = [forest, neigh, elastic, ridge, lasso]
model_names = ["forest", "knn", "elastic", "ridge", "lasso"]

for model in range(len(models)):
    regr = Pipeline([('scaler', StandardScaler()), (model_names[model], models[model])])
    score = cross_val_score(regr, X, y, cv=3, scoring = 'r2').mean()
    print(model_names[model], " score: ", score)    
'''

'\nforest = RandomForestRegressor(max_depth=3, random_state=0)\nkernel = KernelRidge()\nelastic = ElasticNet(random_state=0)\nridge = Ridge(random_state=0)\nlasso = Lasso(random_state=0)\nneigh = KNeighborsRegressor(n_neighbors=2)\n\nmodels = [forest, neigh, elastic, ridge, lasso]\nmodel_names = ["forest", "knn", "elastic", "ridge", "lasso"]\n\nfor model in range(len(models)):\n    regr = Pipeline([(\'scaler\', StandardScaler()), (model_names[model], models[model])])\n    score = cross_val_score(regr, X, y, cv=3, scoring = \'r2\').mean()\n    print(model_names[model], " score: ", score)    \n'

In [ ]:
'''
X_ = X[:20000]
y_ = y[:20000]

kernels = ['rbf', 'poly', 'sigmoid']
cs = [.1, 1, 10]
epsilons = [.001, .01, .1]

for i in range(3):
    for j in range(3):
        for k in range(3):
            svr = SVR(kernel = kernels[i], C = cs[j], epsilon = epsilons[k])
            regr = Pipeline([('scaler', StandardScaler()), ('svr', svr)])
            score = cross_val_score(regr, X_, y_, cv=3, scoring='r2').mean()
            print(kernels[i], " ", cs[j], " ", epsilons[k], ": ", score)
'''

'\nX_ = X[:20000]\ny_ = y[:20000]\n\nkernels = [\'rbf\', \'poly\', \'sigmoid\']\ncs = [.1, 1, 10]\nepsilons = [.001, .01, .1]\n\nfor i in range(3):\n    for j in range(3):\n        for k in range(3):\n            svr = SVR(kernel = kernels[i], C = cs[j], epsilon = epsilons[k])\n            regr = Pipeline([(\'scaler\', StandardScaler()), (\'svr\', svr)])\n            score = cross_val_score(regr, X_, y_, cv=3, scoring=\'r2\').mean()\n            print(kernels[i], " ", cs[j], " ", epsilons[k], ": ", score)\n'

In [ ]:
'''
cs = [5, 10, 25, 50, 75, 100]

for i in range(6):
    svr = SVR(kernel = "rbf", C = cs[i], epsilon = .001)
    regr = Pipeline([('scaler', StandardScaler()), ('svr', svr)])
    score = cross_val_score(regr, X_, y_, cv=3, scoring='r2').mean()
    print(cs[i], ": ", score)
'''

'\ncs = [5, 10, 25, 50, 75, 100]\n\nfor i in range(6):\n    svr = SVR(kernel = "rbf", C = cs[i], epsilon = .001)\n    regr = Pipeline([(\'scaler\', StandardScaler()), (\'svr\', svr)])\n    score = cross_val_score(regr, X_, y_, cv=3, scoring=\'r2\').mean()\n    print(cs[i], ": ", score)\n'

In [ ]:
'''
X_y = pd.concat([X, y], axis=1)
X_y = X_y.sample(400000)
X_ = X_y.drop(["trip_duration"], axis=1)
y_ = X_y["trip_duration"]

svr = SVR(kernel = "rbf", C = 100, epsilon= .001)
regr = Pipeline([('scaler', StandardScaler()), ('svr', svr)])
cross_val_score(regr, X_, y_, cv=2, scoring='r2').mean()
'''
# .019

'\nX_y = pd.concat([X, y], axis=1)\nX_y = X_y.sample(400000)\nX_ = X_y.drop(["trip_duration"], axis=1)\ny_ = X_y["trip_duration"]\n\nsvr = SVR(kernel = "rbf", C = 100, epsilon= .001)\nregr = Pipeline([(\'scaler\', StandardScaler()), (\'svr\', svr)])\ncross_val_score(regr, X_, y_, cv=2, scoring=\'r2\').mean()\n'

In [ ]:
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_score

'''
X_y = pd.concat([X, y], axis=1)
svr1 = SVR()
svr2 = SVR()
svr3 = SVR()
svr4 = SVR()
svr5 = SVR()
svr6 = SVR()
svr7 = SVR()
svr8 = SVR()
svr9 = SVR()
svr10 = SVR() 

svrs = [svr1, svr2, svr3, svr4, svr5, svr6, svr7, svr8, svr9, svr10]

for i in range(10):
    curr_data = X_y.sample(40000)
    X_ = curr_data.drop(["trip_duration"], axis=1)
    y_ = curr_data["trip_duration"]
    regr = Pipeline([('scaler', StandardScaler()), ('svr', svrs[i])])
    regr.fit(X_, y_)
    score = cross_val_score(regr, X_, y_, cv=3, scoring='r2').mean()
    print("model ", i, " score: ", score)
'''

'\nX_y = pd.concat([X, y], axis=1)\nsvr1 = SVR()\nsvr2 = SVR()\nsvr3 = SVR()\nsvr4 = SVR()\nsvr5 = SVR()\nsvr6 = SVR()\nsvr7 = SVR()\nsvr8 = SVR()\nsvr9 = SVR()\nsvr10 = SVR() \n\nsvrs = [svr1, svr2, svr3, svr4, svr5, svr6, svr7, svr8, svr9, svr10]\n\nfor i in range(10):\n    curr_data = X_y.sample(40000)\n    X_ = curr_data.drop(["trip_duration"], axis=1)\n    y_ = curr_data["trip_duration"]\n    regr = Pipeline([(\'scaler\', StandardScaler()), (\'svr\', svrs[i])])\n    regr.fit(X_, y_)\n    score = cross_val_score(regr, X_, y_, cv=3, scoring=\'r2\').mean()\n    print("model ", i, " score: ", score)\n'

In [ ]:
'''
from sklearn.ensemble import VotingRegressor
er = VotingRegressor([('svr1', svr1), ('svr2', svr2), ('svr3' svr3), ('svr4' svr4), ('svr5', svr5), ('svr6', svr6), 
                                                      ('svr7', svr7), ('svr8', svr8), ('svr9', svr9), ('svr10', svr10)])
score = cross_val_score(regr, X)
'''

"\nfrom sklearn.ensemble import VotingRegressor\ner = VotingRegressor([('svr1', svr1), ('svr2', svr2), ('svr3' svr3), ('svr4' svr4), ('svr5', svr5), ('svr6', svr6), \n                                                      ('svr7', svr7), ('svr8', svr8), ('svr9', svr9), ('svr10', svr10)])\nscore = cross_val_score(regr, X)\n"

In [ ]:
#from sklearn.ensemble import BaggingRegressor
#bag = BaggingRegressor(base_estimator=SVR(), n_estimators=20, random_state=0, max_samples=.025, verbose=1, n_jobs=4)
#regr = Pipeline([('scaler', StandardScaler()), ('svr', bag)])
#score = cross_val_score(regr, X, y, cv=2, scoring='r2').mean()

In [ ]:
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

svrs = []
X_y = pd.concat([X, y], axis=1)
for i in range(20):
    X_y_ = X_y.sample(20000)
    X_ = X_y_.drop(['trip_duration'], axis=1)
    y_ = X_y_['trip_duration']
    svr = SVR(kernel = "rbf", C = 100, epsilon= .001)
    regr = Pipeline([('scaler', StandardScaler()), ('svr', svr)])
    regr.fit(X_, y_)
    svrs.append(regr)

In [ ]:
submission = pd.read_csv("../input/nyc-taxi-trip-duration/sample_submission.zip", compression="zip")

for i in range(len(submission)):
    data = test_df.iloc[i, :]
    pred = 0
    for j in range(len(svrs)):
        pred += svrs[j].predict(data.to_numpy().reshape(1, -1))
    submission.iloc[i, 1] = pred / 20
    
submission = submission.set_index('id')
submission.to_csv("sub.csv")